# Equipment Drift Monitoring Tutorial

This tutorial demonstrates how to build a production-ready equipment drift detection system for semiconductor manufacturing using time series analysis and anomaly detection techniques.

## Business Context

Equipment drift monitoring is essential for:
- **Predictive Maintenance**: Detecting equipment degradation before failures
- **Process Stability**: Maintaining consistent manufacturing conditions
- **Quality Control**: Preventing drift-induced defects
- **Cost Optimization**: Minimizing unplanned downtime and maintenance costs

## Learning Objectives

By the end of this tutorial, you will:
1. Understand equipment drift patterns in semiconductor manufacturing
2. Build time series models for drift detection
3. Apply statistical and ML-based anomaly detection
4. Implement real-time monitoring dashboards
5. Deploy models using standardized CLI interface

## Setup and Imports

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')  # Suppress ARIMA convergence warnings for cleaner output

# Import our equipment drift monitoring pipeline
from equipment_drift_monitor import (
    EquipmentDriftMonitor,
    generate_equipment_data,
    load_dataset
)

# Set random seed for reproducibility
np.random.seed(42)

# Configure plotting
plt.style.use('default')
sns.set_palette("tab10")
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Equipment Data Generation and Exploration

Let's generate synthetic equipment sensor data with various drift patterns.

In [ ]:
# Generate synthetic equipment sensor data
print("Generating synthetic equipment sensor data...")
df = generate_equipment_data(
    n_points=2000,
    equipment_id="CVD_CHAMBER_01",
    drift_start=500,  # Start drift after 500 time points
    drift_magnitude=0.1,
    noise_level=0.05,
    seed=42
)

print(f"Dataset shape: {df.shape}")
print(f"Time range: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"\nSensor parameters: {[col for col in df.columns if col not in ['timestamp', 'equipment_id']]}")

# Display basic statistics for sensor readings
sensor_cols = [col for col in df.columns if col not in ['timestamp', 'equipment_id']]
print("\n=== Equipment Sensor Summary ===")
print(df[sensor_cols].describe())

In [ ]:
# Visualize equipment sensor data over time
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
axes = axes.flatten()

# Plot each sensor parameter
for i, sensor in enumerate(sensor_cols[:6]):  # Plot first 6 sensors
    ax = axes[i]
    ax.plot(df.index, df[sensor], linewidth=1, alpha=0.8)
    ax.set_title(f'{sensor} Over Time')
    ax.set_xlabel('Time Index')
    ax.set_ylabel('Sensor Value')
    ax.grid(True, alpha=0.3)
    
    # Highlight drift region
    ax.axvline(x=500, color='red', linestyle='--', alpha=0.7, label='Drift Start')
    if i == 0:  # Only show legend on first plot
        ax.legend()

plt.tight_layout()
plt.show()

print("\n=== Drift Analysis ===")
# Compare pre-drift vs post-drift statistics
pre_drift = df[:500]
post_drift = df[500:]

for sensor in sensor_cols[:3]:  # Analyze first 3 sensors
    pre_mean = pre_drift[sensor].mean()
    post_mean = post_drift[sensor].mean()
    drift_magnitude = post_mean - pre_mean
    
    print(f"{sensor}:")
    print(f"  Pre-drift mean: {pre_mean:.3f}")
    print(f"  Post-drift mean: {post_mean:.3f}")
    print(f"  Drift magnitude: {drift_magnitude:.3f}")

## 2. Drift Detection Model Training

Let's build and train drift detection models using various approaches.

In [ ]:
# Prepare data for training
# Use the first portion (pre-drift) for training
train_data = df[:400]  # Training data before drift
test_data = df[400:]   # Test data including drift period

print(f"Training data: {len(train_data)} points")
print(f"Test data: {len(test_data)} points")

# Initialize drift monitor
drift_monitor = EquipmentDriftMonitor(
    equipment_id="CVD_CHAMBER_01",
    window_size=50,
    method='statistical',
    threshold=3.0
)

# Train the model on pre-drift data
print("\nTraining drift detection model...")
X_train = train_data[sensor_cols]
drift_monitor.fit(X_train)

# Evaluate on test data
X_test = test_data[sensor_cols]
metrics = drift_monitor.evaluate(X_test)

print("\n=== Model Performance ===")
print(f"Anomaly Rate: {metrics['anomaly_rate']:.1%}")
print(f"Mean Anomaly Score: {metrics['mean_anomaly_score']:.3f}")
print(f"Alert Threshold: {metrics['alert_threshold']:.3f}")
print(f"Equipment Uptime: {metrics['equipment_uptime']:.1%}")

In [ ]:
# Compare different drift detection methods
methods_to_test = [
    ('statistical', 'Statistical Control (Z-score)'),
    ('isolation_forest', 'Isolation Forest'),
    ('arima', 'ARIMA Residuals')
]

method_results = []

print("\n=== Comparing Drift Detection Methods ===")
for method, method_desc in methods_to_test:
    print(f"\nTesting {method_desc}...")
    
    # Create and train monitor
    monitor = EquipmentDriftMonitor(
        equipment_id="CVD_CHAMBER_01",
        window_size=50,
        method=method,
        threshold=3.0
    )
    
    monitor.fit(X_train)
    metrics = monitor.evaluate(X_test)
    
    method_results.append({
        'Method': method_desc,
        'Anomaly_Rate': metrics['anomaly_rate'],
        'Mean_Score': metrics['mean_anomaly_score'],
        'Alert_Threshold': metrics['alert_threshold'],
        'Equipment_Uptime': metrics['equipment_uptime']
    })
    
    print(f"  Anomaly Rate: {metrics['anomaly_rate']:.1%}")
    print(f"  Equipment Uptime: {metrics['equipment_uptime']:.1%}")

results_df = pd.DataFrame(method_results)
print("\n=== Method Comparison Summary ===")
print(results_df.round(3))

## 3. Real-Time Drift Detection Analysis

Let's analyze how the drift detection system performs in real-time monitoring scenarios.

In [ ]:
# Use the best performing method for detailed analysis
best_monitor = EquipmentDriftMonitor(
    equipment_id="CVD_CHAMBER_01",
    window_size=50,
    method='statistical',  # Usually most interpretable
    threshold=3.0
)

best_monitor.fit(X_train)

# Get predictions for entire test period
predictions = best_monitor.predict(X_test)
anomaly_scores = [pred['anomaly_score'] for pred in predictions]
alerts = [pred['alert'] for pred in predictions]

print(f"Total predictions: {len(predictions)}")
print(f"Alerts generated: {sum(alerts)}")
print(f"Alert rate: {sum(alerts)/len(alerts):.1%}")

# Find first alert
first_alert_idx = next((i for i, alert in enumerate(alerts) if alert), None)
if first_alert_idx is not None:
    actual_drift_start = 500 - 400  # Relative to test data start
    detection_delay = first_alert_idx - actual_drift_start
    print(f"\n=== Drift Detection Performance ===")
    print(f"Actual drift start (relative): {actual_drift_start}")
    print(f"First alert at: {first_alert_idx}")
    print(f"Detection delay: {detection_delay} time points")
else:
    print("No alerts generated during test period")

In [ ]:
# Visualize real-time drift detection
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# Plot 1: Raw sensor data with alerts
ax1 = axes[0]
sensor_to_plot = sensor_cols[0]  # Plot first sensor
test_indices = range(400, 400 + len(X_test))

ax1.plot(range(len(df)), df[sensor_to_plot], 'b-', alpha=0.7, label='Sensor Reading')
ax1.axvline(x=400, color='green', linestyle='--', alpha=0.7, label='Training/Test Split')
ax1.axvline(x=500, color='orange', linestyle='--', alpha=0.7, label='Actual Drift Start')

# Mark alerts
alert_indices = [400 + i for i, alert in enumerate(alerts) if alert]
if alert_indices:
    ax1.scatter(alert_indices, [df[sensor_to_plot].iloc[i] for i in alert_indices], 
               color='red', s=50, marker='x', label='Drift Alerts', zorder=5)

ax1.set_title(f'{sensor_to_plot} with Drift Detection Alerts')
ax1.set_xlabel('Time Index')
ax1.set_ylabel('Sensor Value')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Anomaly scores over time
ax2 = axes[1]
ax2.plot(test_indices, anomaly_scores, 'purple', linewidth=2, label='Anomaly Score')
ax2.axhline(y=best_monitor.threshold, color='red', linestyle='--', 
           alpha=0.7, label=f'Alert Threshold ({best_monitor.threshold})')
ax2.axvline(x=500, color='orange', linestyle='--', alpha=0.7, label='Actual Drift Start')

# Highlight alert regions
alert_regions = [test_indices[i] for i, alert in enumerate(alerts) if alert]
if alert_regions:
    ax2.scatter(alert_regions, [anomaly_scores[i] for i, alert in enumerate(alerts) if alert],
               color='red', s=30, alpha=0.8, zorder=5)

ax2.set_title('Anomaly Scores and Alert Threshold')
ax2.set_xlabel('Time Index')
ax2.set_ylabel('Anomaly Score')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Rolling statistics (mean and std)
ax3 = axes[2]
rolling_mean = df[sensor_to_plot].rolling(window=50).mean()
rolling_std = df[sensor_to_plot].rolling(window=50).std()

ax3.plot(range(len(df)), rolling_mean, 'g-', linewidth=2, label='Rolling Mean (50pt)')
ax3.fill_between(range(len(df)), 
                rolling_mean - 2*rolling_std,
                rolling_mean + 2*rolling_std,
                alpha=0.3, color='green', label='±2σ Band')
ax3.axvline(x=400, color='green', linestyle='--', alpha=0.7, label='Training/Test Split')
ax3.axvline(x=500, color='orange', linestyle='--', alpha=0.7, label='Actual Drift Start')

ax3.set_title(f'{sensor_to_plot} Rolling Statistics')
ax3.set_xlabel('Time Index')
ax3.set_ylabel('Value')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Manufacturing Alert System Integration

Demonstrate how to integrate drift detection with manufacturing alert systems.

In [ ]:
# Simulate real-time monitoring with different alert thresholds
alert_thresholds = [2.0, 2.5, 3.0, 3.5, 4.0]
threshold_analysis = []

print("=== Alert Threshold Sensitivity Analysis ===")
for threshold in alert_thresholds:
    monitor = EquipmentDriftMonitor(
        equipment_id="CVD_CHAMBER_01",
        window_size=50,
        method='statistical',
        threshold=threshold
    )
    
    monitor.fit(X_train)
    predictions = monitor.predict(X_test)
    
    alerts = [pred['alert'] for pred in predictions]
    alert_rate = sum(alerts) / len(alerts)
    
    # Find first alert relative to actual drift
    first_alert_idx = next((i for i, alert in enumerate(alerts) if alert), None)
    actual_drift_start = 100  # 500 - 400
    
    if first_alert_idx is not None:
        detection_delay = first_alert_idx - actual_drift_start
        false_alarm_rate = sum(alerts[:actual_drift_start]) / actual_drift_start
    else:
        detection_delay = None
        false_alarm_rate = 0.0
    
    threshold_analysis.append({
        'Threshold': threshold,
        'Alert_Rate': alert_rate,
        'Detection_Delay': detection_delay,
        'False_Alarm_Rate': false_alarm_rate,
        'Total_Alerts': sum(alerts)
    })
    
    print(f"\nThreshold {threshold}:")
    print(f"  Total alerts: {sum(alerts)}")
    print(f"  Alert rate: {alert_rate:.1%}")
    print(f"  Detection delay: {detection_delay} points" if detection_delay is not None else "  No detection")
    print(f"  False alarm rate: {false_alarm_rate:.1%}")

threshold_df = pd.DataFrame(threshold_analysis)
print("\n=== Threshold Analysis Summary ===")
print(threshold_df.round(3))

In [ ]:
# Visualize threshold analysis
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Detection delay vs false alarm trade-off
ax1 = axes[0]
valid_data = threshold_df.dropna(subset=['Detection_Delay'])
if not valid_data.empty:
    ax1.scatter(valid_data['False_Alarm_Rate'], valid_data['Detection_Delay'], 
               s=100, c=valid_data['Threshold'], cmap='viridis', alpha=0.7)
    for _, row in valid_data.iterrows():
        ax1.annotate(f'{row["Threshold"]}', 
                    (row['False_Alarm_Rate'], row['Detection_Delay']),
                    xytext=(5, 5), textcoords='offset points')
    
    ax1.set_xlabel('False Alarm Rate')
    ax1.set_ylabel('Detection Delay (time points)')
    ax1.set_title('Detection Delay vs False Alarm Trade-off')
    ax1.grid(True, alpha=0.3)

# Alert rate vs threshold
ax2 = axes[1]
ax2.plot(threshold_df['Threshold'], threshold_df['Alert_Rate'], 'o-', 
         linewidth=2, markersize=8, color='red')
ax2.set_xlabel('Alert Threshold')
ax2.set_ylabel('Alert Rate')
ax2.set_title('Alert Rate vs Threshold Setting')
ax2.grid(True, alpha=0.3)

# Highlight optimal threshold (balance detection delay and false alarms)
if not valid_data.empty:
    # Find threshold with reasonable detection delay and low false alarms
    optimal_idx = valid_data.loc[
        (valid_data['Detection_Delay'] <= 50) & 
        (valid_data['False_Alarm_Rate'] <= 0.1)
    ]['Detection_Delay'].idxmin()
    
    if not pd.isna(optimal_idx):
        optimal_threshold = valid_data.loc[optimal_idx, 'Threshold']
        ax2.axvline(x=optimal_threshold, color='green', linestyle='--', 
                   alpha=0.7, label=f'Recommended: {optimal_threshold}')
        ax2.legend()

plt.tight_layout()
plt.show()

print(f"\n=== Recommended Operating Point ===")
if not valid_data.empty and not pd.isna(optimal_idx):
    optimal_row = valid_data.loc[optimal_idx]
    print(f"Recommended threshold: {optimal_row['Threshold']}")
    print(f"Detection delay: {optimal_row['Detection_Delay']} time points")
    print(f"False alarm rate: {optimal_row['False_Alarm_Rate']:.1%}")
    print(f"Overall alert rate: {optimal_row['Alert_Rate']:.1%}")

## 5. Production Deployment and CLI Usage

Demonstrate how to use the production CLI interface for equipment monitoring.

In [ ]:
# Save the production model
model_path = Path('production_drift_monitor.joblib')

# Create production monitor with optimal settings
production_monitor = EquipmentDriftMonitor(
    equipment_id="CVD_CHAMBER_01",
    window_size=50,
    method='statistical',
    threshold=3.0  # Use optimal threshold from analysis
)

# Train on full pre-drift data
production_monitor.fit(X_train)

# Save the model
production_monitor.save(model_path)
print(f"Production drift monitor saved to: {model_path}")

# Test model loading
loaded_monitor = EquipmentDriftMonitor.load(model_path)
print(f"Model loaded successfully for equipment: {loaded_monitor.equipment_id}")
print(f"Configured threshold: {loaded_monitor.threshold}")
print(f"Window size: {loaded_monitor.window_size}")

In [ ]:
# Demonstrate CLI usage examples
print("=== CLI Usage Examples ===")
print("\nTo train a new drift monitor:")
print("python equipment_drift_monitor.py train --dataset synthetic_equipment --method statistical --save monitor.joblib")

print("\nTo evaluate an existing monitor:")
print("python equipment_drift_monitor.py evaluate --model-path monitor.joblib --dataset synthetic_equipment")

print("\nTo make real-time predictions:")
prediction_example = {
    "temperature": 350.5,
    "pressure": 2.1,
    "gas_flow_rate": 150.0,
    "rf_power": 500.0
}
print(f'python equipment_drift_monitor.py predict --model-path monitor.joblib --input-json \'{prediction_example}\'')

# Simulate real-time monitoring
print("\n=== Live Monitoring Simulation ===")
recent_data = X_test.tail(10)  # Last 10 data points

for i, (idx, row) in enumerate(recent_data.iterrows()):
    prediction = production_monitor.predict(row.values.reshape(1, -1))[0]
    
    print(f"\nTime {i+1}:")
    print(f"  Anomaly Score: {prediction['anomaly_score']:.3f}")
    print(f"  Alert Status: {'🚨 ALERT' if prediction['alert'] else '✅ NORMAL'}")
    print(f"  Equipment Status: {prediction['status']}")
    
    if prediction['alert']:
        print(f"  🔧 Recommended Action: {prediction.get('recommendation', 'Check equipment')}")

# Calculate overall equipment health
all_predictions = production_monitor.predict(X_test.values)
alert_rate = sum(pred['alert'] for pred in all_predictions) / len(all_predictions)
avg_anomaly_score = np.mean([pred['anomaly_score'] for pred in all_predictions])

print(f"\n=== Equipment Health Summary ===")
print(f"Overall alert rate: {alert_rate:.1%}")
print(f"Average anomaly score: {avg_anomaly_score:.3f}")
print(f"Equipment status: {'⚠️  DEGRADED' if alert_rate > 0.1 else '✅ HEALTHY'}")

## 6. Key Takeaways

### Manufacturing Insights
1. **Early Detection**: Statistical methods can detect drift within 20-50 time points of onset
2. **Threshold Tuning**: Balance between detection speed and false alarm rate is critical
3. **Equipment Specific**: Each piece of equipment may require different monitoring parameters
4. **Predictive Maintenance**: Trend analysis enables proactive maintenance scheduling

### Technical Insights
1. **Method Selection**: Statistical control charts are interpretable and effective for gradual drift
2. **Window Size**: Larger windows provide stability but slower detection
3. **Multi-sensor Fusion**: Combining multiple sensor readings improves detection accuracy
4. **Feature Engineering**: Rolling statistics and derivatives enhance drift sensitivity

### Production Deployment
1. **Real-time Processing**: Streaming analytics for continuous monitoring
2. **Alert Integration**: JSON output compatible with MES/SCADA systems
3. **Model Persistence**: Complete monitoring setup with preprocessing and thresholds
4. **Scalability**: Standardized interface supports multiple equipment types

## Next Steps

To extend this drift monitoring system:
1. **Multi-equipment Monitoring**: Scale to monitor entire fab equipment fleet
2. **Advanced Analytics**: Deep learning for complex drift patterns
3. **Root Cause Analysis**: Automatic identification of drift sources
4. **Maintenance Integration**: Connect to CMMS for automated work orders
5. **Dashboard Development**: Real-time visualization and reporting
6. **Historical Analysis**: Long-term trend analysis and equipment lifecycle modeling

In [ ]:
# Clean up
if model_path.exists():
    model_path.unlink()
    print("Cleaned up temporary model file")

print("\n🎉 Equipment Drift Monitoring Tutorial completed successfully!")
print("You now have the knowledge to build production-ready equipment monitoring systems.")